In [ ]:
from collections import Counter
import json
from sklearn.cluster import KMeans
import pandas
import matplotlib.pyplot as plt
import pprint
from nltk.corpus import stopwords
import string
from nltk.tokenize import TweetTokenizer
from wordcloud import WordCloud

In [ ]:
tweets = []

with open('result.txt','rt') as tweetFile:
    for line in tweetFile:
        try:
            tweets.append(json.loads(line))
        except:
            pass

In [ ]:
tData = pandas.DataFrame.from_dict(tweets)

In [ ]:
excludeWord = stopwords.words("English") + \
                list(string.punctuation) + \
                ['rt','via','...','…','“','”','️','’','https','⚡',"it's","don't",'thanks', \
                 'w','4','4.0','2','80','https://t.…']
tTokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)

In [ ]:
tokenList = []
for item in tData['text']:
    try:
        tokenList.append([word for word in tTokenizer.tokenize(item) if word not in excludeWord])
    except:
        tokenList.append([])
tData['token'] = tokenList

In [ ]:
tokenCount = Counter()
for item in tData['token']:
    tokenCount.update(item)

In [ ]:
token100 = [item[0] for item in tokenCount.most_common(100)]

In [ ]:
tokenVec = pandas.DataFrame(columns = token100)
for token in token100:
    tokenVec[token] = [int(token in item) for item in tData['token']]

In [ ]:
n = 3
kmeans1 = KMeans(n_clusters = n).fit(tokenVec)
pre1 = kmeans1.predict(tokenVec)

In [ ]:
for i in range(n):
    saveFilename = 'cluster' + str(i) + 'of' + str(n) + '.txt'
    with open(saveFilename,'wt') as saveFile:
        for item in tData[pre1==i]['token']:
            for word in item:
                try:
                    saveFile.write(word + ',')
                except:
                    pass
    
    text = open(saveFilename,'rt').read()
    wCloud = WordCloud().generate(text)
    plt.imshow(wCloud)
    plt.axis("off")
    plt.show()

In [ ]:
preCount = Counter()
preCount.update(pre1)
preCount

In [ ]:
group1Vec = tokenVec[pre1==1]

In [ ]:
m = 4
kmeans2 = KMeans(n_clusters = m).fit(group1Vec)

In [ ]:
pre2 = kmeans2.predict(group1Vec)

In [ ]:
group1Data = tData[pre1==1]

In [ ]:
for i in range(m):
    saveFilename = 'clusterGroup1_' + str(i) + 'of' + str(n) + '.txt'
    with open(saveFilename,'wt') as saveFile:
        for item in group1Data[pre2==i]['token']:
            for word in item:
                try:
                    saveFile.write(word + ',')
                except:
                    pass
    
    text = open(saveFilename,'rt').read()
    wCloud = WordCloud().generate(text)
    plt.imshow(wCloud)
    plt.axis("off")
    plt.show()

In [ ]:
level2Count = Counter()
level2Count.update(pre2)
level2Count

In [ ]:
with open('AllText.txt','wt') as saveFile:
    for item in tData['text']:
        try:
            saveFile.write(item + '\n')
        except:
            pass